Neighbourhood Analysis for New Business

A startup or a businessman who wants to open a shop has a series of things to take care to be successful with his business. He must be an experienced person in doing business in his respective field with respect to the day to day activities that he needs to carry out. But in order to chose the right location for his business to flourish and analyze the potential growth a data scientist can help him with the data and analyze it to achieve practical outcomes.

In this capstone project, the analysis is done to help a person who wants to open a spa in the right location so that the footfall is maximum. The spa business is affected by a lot of factors such as the quality of service, the kind of chemicals used, the brand name of the business etc. These are the factors that an individual businessman needs to provide. Sometimes inspite of providing the best of service the footfall in a shop is less due to the location and the neighbourhood shops. In order to help the business choose the right location the current neighbourhoods need to be analyzed.

The dataset used for the project is downloaded from  https://geo.nyu.edu/catalog/nyu_2451_34572

In [ ]:
Import all libraries and dependencies

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Download the dataset from https://geo.nyu.edu/catalog/nyu_2451_34572 for New York city

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']    

Put the data into a dataframe

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)    

The dataframe has 5 boroughs and 306 neighborhoods.


In [5]:
#display the different boroughs in New York city
neighborhoods['Borough'].unique().tolist()

['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island']

SLicing Data for Brooklyn Borough

In [6]:
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)

In [7]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

# create map of Brooklyn using latitude and longitude values
map_Brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Brooklyn)  
    
map_Brooklyn

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


Four API credentials

In [8]:
CLIENT_ID = 'TNDOMNP5U2KCIWYXCNNR5OM4POAZXPE3GEYDRSEIUJVQXGR0' # Foursquare ID
CLIENT_SECRET = 'XWXLK2HCOSSGIKJE2CI2MFO4AKBJUSJ1SKXLJG2O3JBDFQFK' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: TNDOMNP5U2KCIWYXCNNR5OM4POAZXPE3GEYDRSEIUJVQXGR0
CLIENT_SECRET:XWXLK2HCOSSGIKJE2CI2MFO4AKBJUSJ1SKXLJG2O3JBDFQFK


In [9]:
#display the different neighbourhood in Brooklyn Borough
Brooklyn_data['Neighborhood'].unique().tolist()

['Bay Ridge',
 'Bensonhurst',
 'Sunset Park',
 'Greenpoint',
 'Gravesend',
 'Brighton Beach',
 'Sheepshead Bay',
 'Manhattan Terrace',
 'Flatbush',
 'Crown Heights',
 'East Flatbush',
 'Kensington',
 'Windsor Terrace',
 'Prospect Heights',
 'Brownsville',
 'Williamsburg',
 'Bushwick',
 'Bedford Stuyvesant',
 'Brooklyn Heights',
 'Cobble Hill',
 'Carroll Gardens',
 'Red Hook',
 'Gowanus',
 'Fort Greene',
 'Park Slope',
 'Cypress Hills',
 'East New York',
 'Starrett City',
 'Canarsie',
 'Flatlands',
 'Mill Island',
 'Manhattan Beach',
 'Coney Island',
 'Bath Beach',
 'Borough Park',
 'Dyker Heights',
 'Gerritsen Beach',
 'Marine Park',
 'Clinton Hill',
 'Sea Gate',
 'Downtown',
 'Boerum Hill',
 'Prospect Lefferts Gardens',
 'Ocean Hill',
 'City Line',
 'Bergen Beach',
 'Midwood',
 'Prospect Park South',
 'Georgetown',
 'East Williamsburg',
 'North Side',
 'South Side',
 'Ocean Parkway',
 'Fort Hamilton',
 'Ditmas Park',
 'Wingate',
 'Rugby',
 'Remsen Village',
 'New Lots',
 'Paerdegat Ba

Function to explore the neighbouhoods in Brooklyn

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    LIMIT=100
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
#Finding the venues for each neighbourhood in Brooklyn
Brooklyn_venues = getNearbyVenues(names=Brooklyn_data['Neighborhood'],
                                   latitudes=Brooklyn_data['Latitude'],
                                   longitudes=Brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [12]:
Brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,51,51,51,51,51,51
Bay Ridge,89,89,89,89,89,89
Bedford Stuyvesant,27,27,27,27,27,27
Bensonhurst,31,31,31,31,31,31
Bergen Beach,7,7,7,7,7,7
Boerum Hill,83,83,83,83,83,83
Borough Park,20,20,20,20,20,20
Brighton Beach,43,43,43,43,43,43
Broadway Junction,18,18,18,18,18,18


In [13]:
# one hot encoding
Brooklyn_onehot = pd.get_dummies(Brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Brooklyn_onehot['Neighborhood'] = Brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]

Brooklyn_onehot.head()

,Yoga Studio,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,V

In [14]:
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
Brooklyn_grouped

,Neighborhood,Yoga Studio,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,V

In [15]:
for hood in Brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Brooklyn_grouped[Brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True))
    print('\n')

----Bath Beach----
                                        venue  freq
0                                    Pharmacy  0.06
1                          Chinese Restaurant  0.06
2                                  Donut Shop  0.06
3                                 Pizza Place  0.06
4                                 Video Store  0.04
5                            Sushi Restaurant  0.04
6                                  Kids Store  0.04
7                          Italian Restaurant  0.04
8                                  Restaurant  0.04
9                                  Shoe Store  0.04
10                       Fast Food Restaurant  0.04
11                               Burger Joint  0.02
12                            Bubble Tea Shop  0.02
13                               Dessert Shop  0.02
14                                Coffee Shop  0.02
15                             Clothing Store  0.02
16                       Pakistani Restaurant  0.02
17                                  Surf Spot

Sort venues in descending order

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 30

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,Bath Beach,Pizza Place,Donut Shop,Pharmacy,Chinese Restaurant,Restaurant,Italian Restaurant,Sushi Restaurant,Shoe Store,Video Store,Kids Store,Fast Food Restaurant,Pakistani Restaurant,Flower Shop,Mobile Phone Shop,Cantonese Restaurant,Bubble Tea Shop,Liquor Store,Sandwich Place,Women's Store,Clothing Store,Coffee Shop,Rental Car Location,Hookah Bar,Playground,Dessert Shop,Peruvian Restaurant,Diner,Park,Surf Spot,Burger Joint
1,Bay Ridge,Pizza Place,Spa,Italian Restaurant,Greek Restaurant,Grocery Store,American Restaurant,Chinese Restaurant,Bar,Bagel Shop,Playground,Pharmacy,Sushi Restaurant,Optical Shop,Thai Restaurant,Ice Cream Shop,Seafood Restaurant,Hookah Bar,Sandwich Place,Coffee Shop,Kids Store,Mattress Store,Mexican Restaurant,Lounge,Middle Eastern Restaurant,Bookstore,Breakfast Spot,Snack Place,Pool Hall,Lingerie Store,Clothing Store
2,Bedford Stuyvesant,Coffee Shop,Pizza Place,Bar,Café,Bus Stop,Bagel Shop,Fried Chicken Joint,Vietnamese Restaurant,Thrift / Vintage Store,Gourmet Shop,BBQ Joint,Wine Bar,Juice Bar,Cocktail Bar,Deli / Bodega,Park,Japanese Restaurant,Wine Shop,New American Restaurant,Fruit & Vegetable Store,Boutique,Fish Market,Donut Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant
3,Bensonhurst,Chinese Restaurant,Pizza Place,Sushi Restaurant,Dessert Shop,Ice Cream Shop,Smoke Shop,Bagel Shop,Liquor Store,Coffee Shop,Grocery Store,Cosmetics Shop,Shabu-Shabu Restaurant,Spa,Bakery,Noodle House,Park,Hotpot Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Donut Shop,Italian Restaurant,Butcher,Food & Drink Shop,Field,Food Court,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Hotel
4,Bergen Beach,Harbor / Marina,Baseball Field,Hockey Field,Donut Shop,Playground,Athletics & Sports,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food Court,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fast Food Restaurant,Women's Store,Farmers Market,Gaming Cafe,Farm,Falafel Restaurant,Factory,Event Space,Event Service,Ethiopian Restaurant
5,Boerum Hill,Coffee Shop,Bar,Spa,French Restaurant,Dance Studio,Cosmetics Shop,Furniture / Home Store,Sandwich Place,Martial Arts Dojo,Grocery Store,Chinese Restaurant,Kids Store,Yoga Studio,Athletics & Sports,Middle Eastern Restaurant,Arts & Crafts Store,Bakery,Cocktail Bar,Flower Shop,Boutique,Bubble Tea Shop,Burger Joint,Burrito Place,Music Store,Seafood Restaurant,Liquor Store,Performing Arts Venue,Pharmacy,Department Store,Mexican Restaurant
6,Borough Park,Bank,Pizza Place,Deli / Bodega,Pharmacy,Coffee Shop,Café,Farmers Market,Men's Store,Fast Food Restaurant,Restaurant,Hostel,Hotel,American Restaurant,Grocery Store,Food Stand,Food Court,Food & Drink Shop,Food Truck,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,French Restaurant,Women's Store,Farm,Falafel Restaurant,Frozen Yogurt Shop,Factory
7,Brighton Beach,Beach,Eastern European Restaurant,Restaurant,Pharmacy,Mobile Phone Shop,Gourmet Shop,Bank,Russian Restaurant,Sushi Restaurant,Fast Food Restaurant,Lounge,Bookstore,Taco Place,Grocery Store,Non-Profit,Korean Restaurant,Coffee Shop,Bakery,Convenience Store,Other Great Outdoors,Varenyky restaurant,Mediterranean Restaurant

k-means clustering

In [18]:
# set number of clusters
kclusters = 10

Brooklyn_grouped_clustering = Brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([2, 2, 0, 2, 3, 0, 1, 2, 2, 0, 2, 0, 7, 0, 2, 2, 0, 2, 1, 2, 2, 0,
       0, 6, 9, 2, 0, 2, 2, 9], dtype=int32)

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Brooklyn_merged = Brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Brooklyn_merged = Brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Brooklyn_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,2,Pizza Place,Spa,Italian Restaurant,Greek Restaurant,Grocery Store,American Restaurant,Chinese Restaurant,Bar,Bagel Shop,Playground,Pharmacy,Sushi Restaurant,Optical Shop,Thai Restaurant,Ice Cream Shop,Seafood Restaurant,Hookah Bar,Sandwich Place,Coffee Shop,Kids Store,Mattress Store,Mexican Restaurant,Lounge,Middle Eastern Restaurant,Bookstore,Breakfast Spot,Snack Place,Pool Hall,Lingerie Store,Clothing Store
1,Brooklyn,Bensonhurst,40.611009,-73.995180,2,Chinese Restaurant,Pizza Place,Sushi Restaurant,Dessert Shop,Ice Cream Shop,Smoke Shop,Bagel Shop,Liquor Store,Coffee Shop,Grocery Store,Cosmetics Shop,Shabu-Shabu Restaurant,Spa,Bakery,Noodle House,Park,Hotpot Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Donut Shop,Italian Restaurant,Butcher,Food & Drink Shop,Field,Food Court,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Hotel
2,Brooklyn,Sunset Park,40.645103,-74.010316,2,Bank,Latin American Restaurant,Mexican Restaurant,Bakery,Pizza Place,Gym,Mobile Phone Shop,Pharmacy,Sandwich Place,Supplement Shop,Breakfast Spot,Shoe Store,Women's Store,Record Shop,Grocery Store,Italian Restaurant,Donut Shop,Ice Cream Shop,Bagel Shop,Video Game Store,Hotel,Fish & Chips Shop,Fruit & Vegetable Store,Argentinian Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,Art Gallery,French Restaurant,Food Truck,Food Stand
3,Brooklyn,Greenpoint,40.730201,-73.954241,0,Bar,Pizza Place,Coffee Shop,Café,Cocktail Bar,Mexican Restaurant,Record Shop,Bakery,Yoga Studio,New American Restaurant,Spa,Italian Restaurant,Polish Restaurant,Grocery Store,Furniture / Home Store,French Restaurant,Sandwich Place,Boutique,Salon / Barbershop,Restaurant,Rental Car Location,Dance Studio,Ramen Restaurant,Hotel,Ice Cream Shop,Coworking Space,Gymnastics Gym,Juice Bar,Deli / Bodega,Organic Grocery
4,Brooklyn,Gravesend,40.595260,-73.973471,2,Italian Restaurant,Chinese Restaurant,Lounge,Bakery,Pizza Place,Record Shop,Deli / Bodega,Music Venue,Pharmacy,Music Store,Bus Station,Men's Store,Martial Arts Dojo,Donut Shop,Gift Shop,Grocery Store,Bar,Spa,Wine Shop,Hookah Bar,Automotive Shop,Harbor / Marina,Falafel Restaurant,Hockey Field,Home Service,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant
5,Brooklyn,Brighton Beach,40.576825,-73.965094,2,Beach,Eastern European Restaurant,Restaurant,Pharmacy,Mobile Phone Shop,Gourmet Shop,Bank,Russian Restaurant,Sushi Restaurant,Fast Food Restaurant,Lounge,Bookstore,Taco Place,Grocery Store,Non-Profit,Korean Restaurant,Coffee Shop,Bakery,Convenience Store,Other Great Outdoors,Varenyky restaurant,Mediterranean Restaurant,Donut Shop,Dessert Shop,Food & Drink Shop,Playground,Diner,Supermarket,Fish & Chips Shop,Fish Market
6,Brooklyn,Sheepshead Bay,40.586890,-73.943186,2,Turkish Restaurant,Dessert Shop,Sandwich Place,Italian Restaurant,Yoga Studio,Diner,Russian Restaurant,Restaurant,Creperie,Pizza Place,Buffet,Park,Miscellaneous Shop,Grocery Store,Harbor / Marina,Karaoke Bar,Hotel,Café,Department Store,American Restaurant,Bar,Fish Market,Gaming Cafe,Furniture / Home Store,Fruit & Vegetable Store,Art Gallery,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
7,Brooklyn,Manhattan Terrace,40.614433,-73.957438,1,Pizza Place,Donut Sh

Visualizing the clustering

In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Brooklyn_merged['Latitude'], Brooklyn_merged['Longitude'], Brooklyn_merged['Neighborhood'], Brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examinig the Cluster 4

In [21]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 4]#, Brooklyn_merged.columns[[1] + list(range(10, Brooklyn_merged.shape[1]))]]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
39,Brooklyn,Sea Gate,40.576375,-74.007873,4,Spa,Beach,Optical Shop,Bus Station,Home Service,Women's Store,Food & Drink Shop,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food Court,Field,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Filipino Restaurant,Fast Food Restaurant,Furniture / Home Store,Eastern European Restaurant,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Farmers Market


In [22]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 5]#, Brooklyn_merged.columns[[1] + list(range(10, Brooklyn_merged.shape[1]))]]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
59,Brooklyn,Paerdegat Basin,40.631318,-73.902335,5,Bus Line,Gym,Food,Women's Store,Food & Drink Shop,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Court,Gaming Cafe,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Field,Fast Food Restaurant,Farmers Market,Dumpling Restaurant,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Farm,Electronics Store


In [33]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 6]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
35,Brooklyn,Dyker Heights,40.619219,-74.019314,6,Burger Joint,Golf Course,Dance Studio,Playground,Bagel Shop,Hot Dog Joint,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Food Court,Field,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Filipino Restaurant,Women's Store,Fast Food Restaurant,Farmers Market,Furniture / Home Store,Falafel Restaurant,Factory,Event Space,Event Service,Ethiopian Restaurant,Electronics Store


In [34]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 7]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
28,Brooklyn,Canarsie,40.635564,-73.902093,7,Asian Restaurant,Bus Line,Gym,Grocery Store,Caribbean Restaurant,Women's Store,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Field,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Filipino Restaurant,Fast Food Restaurant,Furniture / Home Store,Eastern European Restaurant,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Farmers Market


In [35]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 8]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
30,Brooklyn,Mill Island,40.606336,-73.908186,8,Pool,Lake,Food,Women's Store,Food & Drink Shop,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Court,Farmers Market,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fast Food Restaurant,Farm,Gaming Cafe,Falafel Restaurant,Factory,Event Space,Event Service,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dry Cleaner


In [36]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 9]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
10,Brooklyn,East Flatbush,40.641718,-73.936103,9,Print Shop,Pharmacy,Check Cashing Service,Caribbean Restaurant,Hardware Store,Park,Food,Chinese Restaurant,Supermarket,Fast Food Restaurant,Deli / Bodega,Department Store,Moving Target,Wine Shop,Food Court,Food Stand,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Food Truck,Field,Food & Drink Shop,Falafel Restaurant,Farmers Market,Farm,Fried Chicken Joint,Factory,Event Space,Event Service
29,Brooklyn,Flatlands,40.630446,-73.929113,9,Fast Food Restaurant,Pharmacy,Caribbean Restaurant,Discount Store,Athletics & Sports,Nightclub,Lounge,Sandwich Place,Bar,Paper / Office Supplies Store,Park,Fried Chicken Joint,Electronics Store,Video Store,Deli / Bodega,Fish & Chips Shop,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Dive Bar,Dog Run,Donut Shop,Filipino Restaurant,Ethiopian Restaurant,Field
55,Brooklyn,Wingate,40.660947,-73.937187,9,Fast Food Restaurant,Bus Station,Juice Bar,Grocery Store,Other Great Outdoors,Gym / Fitness Center,Field,Fish & Chips Shop,BBQ Joint,Rental Car Location,Café,Pharmacy,Donut Shop,Deli / Bodega,Vietnamese Restaurant,Discount Store,Pizza Place,Intersection,Sandwich Place,Food Court,Farmers Market,Food & Drink Shop,Food,Flower Shop,Fish Market,Filipino Restaurant,Food Stand,Women's Store,Farm,Falafel Restaurant
57,Brooklyn,Remsen Village,40.652117,-73.916653,9,Caribbean Restaurant,Fast Food Restaurant,Supermarket,Sandwich Place,Salad Place,Fish Market,Coffee Shop,Deli / Bodega,Donut Shop,Pizza Place,Breakfast Spot,Fried Chicken Joint,Food Truck,Food Stand,Food Court,Field,Food,Flower Shop,French Restaurant,Fish & Chips Shop,Filipino Restaurant,Food & Drink Shop,Farm,Farmers Market,Fruit & Vegetable Store,Falafel Restaurant,Factory,Event Space,Event Service,Ethiopian Restaurant


In [40]:
Brooklyn_merged.loc[Brooklyn_merged['Cluster Labels'] == 3]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
45,Brooklyn,Bergen Beach,40.61515,-73.898556,3,Harbor / Marina,Baseball Field,Hockey Field,Donut Shop,Playground,Athletics & Sports,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food Court,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fast Food Restaurant,Women's Store,Farmers Market,Gaming Cafe,Farm,Falafel Restaurant,Factory,Event Space,Event Service,Ethiopian Restaurant
